In [1]:
from utils import *
import planetary_computer
import stackstac
import rioxarray as rxr
import xarray as xr
from rioxarray.merge import merge_datasets

In [ ]:
from dask.distributed import Client as DaskClient, LocalCluster
cluster = LocalCluster()
client = DaskClient(cluster)
client

In [3]:
from pystac_client import Client as PystacClient
api_url = "https://earth-search.aws.element84.com/v1"
stac_client = PystacClient.open(api_url)

In [4]:
bbox = (-121.45522382606673, 39.9781724447063, -119.99678876747298, 40.81484858063401)

In [5]:
pre_datetime = "2021-07-08"
august_datetime = "2021-08-14"
september_datetime = "2021-09-11"
october_datetime = "2021-10-16" 
post_datetime = "2021-10-28"

In [6]:
pre = FindBoxScene(pre_datetime, bbox)
print(pre)
nir_urls = gen_stac_asset_urls(pre, 'nir')
swir_urls = gen_stac_asset_urls(pre, 'swir16')

print('nir', nir_urls)
print('swir', swir_urls)

/opt/conda/envs/geog213-final/lib/python3.13/site-packages/pystac_client/item_search.py:850: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


8
nir ['https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2A_10TFK_20210708_0_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2A_10TFK_20210708_1_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/GK/2021/7/S2A_10TGK_20210708_1_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/11/T/KE/2021/7/S2A_11TKE_20210708_1_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FL/2021/7/S2A_10TFL_20210708_0_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FL/2021/7/S2A_10TFL_20210708_1_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/GL/2021/7/S2A_10TGL_20210708_0_L2A/B08.tif', 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/GL/2021/7/S2A_10TGL_20210708_1_L2A/B08.tif']
swir ['https://sentinel-c

In [7]:
nir_datasets = [
    rxr.open_rasterio(url, lock=False, chunks=(1, 'auto', -1)).to_dataset(name="nir") for url in nir_urls
]

nir_datasets

[<xarray.Dataset> Size: 241MB
 Dimensions:      (band: 1, x: 10980, y: 10980)
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 88kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
   * y            (y) float64 88kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
     spatial_ref  int64 8B 0
 Data variables:
     nir          (band, y, x) uint16 241MB dask.array<chunksize=(1, 6111, 10980), meta=np.ndarray>,
 <xarray.Dataset> Size: 241MB
 Dimensions:      (band: 1, x: 10980, y: 10980)
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 88kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
   * y            (y) float64 88kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
     spatial_ref  int64 8B 0
 Data variables:
     nir          (band, y, x) uint16 241MB dask.array<chunksize=(1, 6111, 10980), meta=np.ndarray>,
 <xarray.Dataset> Size: 241MB
 Dimensions:      (band: 1, x: 10980, y: 10980)
 Coordinates:
   * band         (band) int64 8B 1
   * x

In [8]:
swir_datasets = [
    rxr.open_rasterio(url, lock=False, chunks=(1, 'auto', -1)).to_dataset(name="swir") for url in swir_urls
]

swir_datasets

[<xarray.Dataset> Size: 60MB
 Dimensions:      (band: 1, x: 5490, y: 5490)
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 44kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
   * y            (y) float64 44kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
     spatial_ref  int64 8B 0
 Data variables:
     swir         (band, y, x) uint16 60MB dask.array<chunksize=(1, 5490, 5490), meta=np.ndarray>,
 <xarray.Dataset> Size: 60MB
 Dimensions:      (band: 1, x: 5490, y: 5490)
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 44kB 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05
   * y            (y) float64 44kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
     spatial_ref  int64 8B 0
 Data variables:
     swir         (band, y, x) uint16 60MB dask.array<chunksize=(1, 5490, 5490), meta=np.ndarray>,
 <xarray.Dataset> Size: 60MB
 Dimensions:      (band: 1, x: 5490, y: 5490)
 Coordinates:
   * band         (band) int64 8B 1
   * x            (

In [ ]:
mosaic_nir = merge_datasets(nir_datasets)
mosaic_swir = merge_datasets(swir_datasets)

In [17]:
print("Mosaic CRS:", mosaic_nir.rio.crs)

Mosaic CRS: EPSG:32610


In [18]:
print("Original mosaic bounds:", mosaic_nir.rio.bounds())

print("bbox_geometry:", bbox_geometry)


Original mosaic bounds: (600000.0, 4387360.0, 824670.0, 4600020.0)
bbox_geometry: {'type': 'Polygon', 'coordinates': [[[-121.45522382606673, 39.9781724447063], [-121.45522382606673, 40.81484858063401], [-119.99678876747298, 40.81484858063401], [-119.99678876747298, 39.9781724447063], [-121.45522382606673, 39.9781724447063]]]}


In [14]:
bbox_geometry = {
    'type': 'Polygon',
    'coordinates': [[
        [-121.45522382606673, 39.9781724447063],
        [-121.45522382606673, 40.81484858063401],
        [-119.99678876747298, 40.81484858063401],
        [-119.99678876747298, 39.9781724447063],
        [-121.45522382606673, 39.9781724447063]
    ]]
}

In [21]:
from pyproj import Transformer
import numpy as np

# Create transformer
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32610", always_xy=True)

# Original coordinates from your bbox_geometry
coords = bbox_geometry['coordinates'][0]

# Transform each coordinate pair
utm_coords = []
for lon, lat in coords:
    x, y = transformer.transform(lon, lat)
    utm_coords.append([x, y])

# Create new geometry dictionary in UTM
bbox_geometry_utm = {
    'type': 'Polygon',
    'coordinates': [utm_coords]
}

# Print transformed coordinates to verify
print("UTM coordinates:", bbox_geometry_utm)

UTM coordinates: {'type': 'Polygon', 'coordinates': [[[631906.4285081971, 4426477.211218987], [630283.2475873799, 4519351.933880895], [753297.2287633473, 4522544.949250258], [756455.5246679303, 4429655.072561484], [631906.4285081971, 4426477.211218987]]]}


In [22]:
mosaic_nir.nir.rio.to_raster("nir_mosaic.tif")
mosaic_nir = mosaic_nir.rio.clip(bbox_geometry, crs=4326)

NoDataInBounds: No data found in bounds. Data variable: nir

In [ ]:
mosaic_swir.swir.rio.to_raster("swir_mosaic.tif")
mosaic_swir = mosaic_swir.rio.clip(bbox, crs=4326